<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#groupBy" data-toc-modified-id="groupBy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>groupBy</a></span><ul class="toc-item"><li><span><a href="#rollup" data-toc-modified-id="rollup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>rollup</a></span></li><li><span><a href="#cube" data-toc-modified-id="cube-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>cube</a></span></li></ul></li><li><span><a href="#pivot" data-toc-modified-id="pivot-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>pivot</a></span></li></ul></div>

In [1]:
spark

25/09/16 15:09:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/09/16 15:09:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [3]:
print(f"spark.executor.cores = {spark.conf.get('spark.executor.cores')}\nspark.executor.memory = {spark.conf.get('spark.executor.memory')}")

spark.executor.cores = 1
spark.executor.memory = 1g


In [4]:
meteo = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('donnees/meteo/2024/') #    .cache()

meteo = meteo.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations') #             .cache()


from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('donnees/postesSynop.csv')  \
      .cache()

meteo.count(), villes.count()

25/09/16 15:09:53 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


(176468, 62)

# groupBy

<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-03.png" width="400">    

In [5]:
meteo.where('id < 8000')\
     .select('annee','mois_jour','temperature','precipitations')\
     .describe().show()

25/09/16 15:09:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 9:=============================>                             (1 + 1) / 2]

+-------+--------------------+-----------------+-----------------+--------------------+
|summary|               annee|        mois_jour|      temperature|      precipitations|
+-------+--------------------+-----------------+-----------------+--------------------+
|  count|              122295|           122295|           119253|              119114|
|   mean|              2024.0|667.2831841040108|13.31191835844796|  0.2815034336853713|
| stddev|1.136676643633269...| 345.406742820554|6.834882732270117|   1.344748843742711|
|    min|                2024|             0101|            -12.1|-0.30000000000000004|
|    max|                2024|             1231|             38.7|                56.9|
+-------+--------------------+-----------------+-----------------+--------------------+



In [6]:
meteo.where('id < 8000')\
     .select('annee','mois_jour','temperature')\
     .describe().show()

[Stage 12:=============================>                            (1 + 1) / 2]

+-------+--------------------+-----------------+-----------------+
|summary|               annee|        mois_jour|      temperature|
+-------+--------------------+-----------------+-----------------+
|  count|              122295|           122295|           119253|
|   mean|              2024.0|667.2831841040108|13.31191835844796|
| stddev|1.136676643633269...| 345.406742820554|6.834882732270117|
|    min|                2024|             0101|            -12.1|
|    max|                2024|             1231|             38.7|
+-------+--------------------+-----------------+-----------------+



In [7]:
meteo.where('id < 8000').count()

122295

In [8]:
meteo.where('id < 8000')\
     .select('humidite','visibilite','pression')\
     .describe().show()

+-------+-------------------+-----------------+------------------+
|summary|           humidite|       visibilite|          pression|
+-------+-------------------+-----------------+------------------+
|  count|             118949|           111751|            118863|
|   mean| 0.7836361802117019|24.83964045064438| 99.53958002069538|
| stddev|0.16311435403412572|16.31514120903404|2.5643461686869444|
|    min|               0.01|              0.0|             88.85|
|    max|                1.0|             80.0|            103.77|
+-------+-------------------+-----------------+------------------+



In [9]:
meteo.where('id < 8000')\
     .groupBy('annee')\
     .avg('temperature','visibilite','pression').show(5)

+-----+-----------------+-----------------+-----------------+
|annee| avg(temperature)|  avg(visibilite)|    avg(pression)|
+-----+-----------------+-----------------+-----------------+
| 2024|13.31191835844796|24.83964045064438|99.53958002069538|
+-----+-----------------+-----------------+-----------------+



In [10]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .max('temperature','visibilite','pression').show(5)

+----+-----+----------------+---------------+-------------+
|  id|annee|max(temperature)|max(visibilite)|max(pression)|
+----+-----+----------------+---------------+-------------+
|7149| 2024|            36.3|           60.0|       102.72|
|7020| 2024|            24.6|           25.0|       103.57|
|7139| 2024|            34.8|           20.0|       102.04|
|7661| 2024|            NULL|           30.0|         NULL|
|7037| 2024|            35.3|           60.0|       101.78|
+----+-----+----------------+---------------+-------------+
only showing top 5 rows


In [11]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            count('id').alias('nb_villes'),
            round(avg('temperature'),2).alias('temperature'),
            round(avg('humidite'),2).alias('humidite'),
            round(avg('visibilite'),2).alias('visibilite'),
            round(avg('pression'),2).alias('pression'),
            round(sum('pression')).alias('precipitations'))\
     .orderBy("id","annee")\
     .show(28)

+----+-----+---------+-----------+--------+----------+--------+--------------+
|  id|annee|nb_villes|temperature|humidite|visibilite|pression|precipitations|
+----+-----+---------+-----------+--------+----------+--------+--------------+
|7005| 2024|     2926|      11.74|    0.83|     16.05|  100.69|      294607.0|
|7015| 2024|     2927|      12.15|    0.81|     21.87|  100.96|      295518.0|
|7020| 2024|     2924|      12.89|    0.83|     12.88|  101.41|      296511.0|
|7027| 2024|     2923|      12.07|    0.83|      23.5|  100.79|      294601.0|
|7037| 2024|     2923|      11.37|    0.84|     24.72|   99.71|      291466.0|
|7072| 2024|     2925|      11.96|    0.83|     16.75|  100.47|      293880.0|
|7110| 2024|     2924|      12.06|    0.86|     20.04|  100.43|      293663.0|
|7117| 2024|     2924|      12.82|    0.83|     15.24|  100.87|      294947.0|
|7130| 2024|     2921|      12.54|    0.82|     24.44|   101.2|      295602.0|
|7139| 2024|     2927|      11.77|    0.83|      16.

In [12]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show(5)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 2024|   1|        1013.0|
| 2024|   2|         938.0|
| 2024|   3|        1001.0|
| 2024|   4|         972.0|
| 2024|   5|        1003.0|
+-----+----+--------------+
only showing top 5 rows


## rollup
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-05.png" width="400">

In [13]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .show(14)

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 2024|   1|        1013.0|
| 2024|   2|         938.0|
| 2024|   3|        1001.0|
| 2024|   4|         972.0|
| 2024|   5|        1003.0|
| 2024|   6|         967.0|
| 2024|   7|        1006.0|
| 2024|   8|        1011.0|
| 2024|   9|         976.0|
| 2024|  10|         998.0|
| 2024|  11|         952.0|
| 2024|  12|         993.0|
| 2024|NULL|       11830.0|
| NULL|NULL|       11830.0|
+-----+----+--------------+



In [14]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .orderBy('annee','mois')\
     .show()

+-----+----+--------------+
|annee|mois|precipitations|
+-----+----+--------------+
| 2024|   1|        1013.0|
| 2024|   2|         938.0|
| 2024|   3|        1001.0|
| 2024|   4|         972.0|
| 2024|   5|        1003.0|
| 2024|   6|         967.0|
| 2024|   7|        1006.0|
| 2024|   8|        1011.0|
| 2024|   9|         976.0|
| 2024|  10|         998.0|
| 2024|  11|         952.0|
| 2024|  12|         993.0|
+-----+----+--------------+



In [15]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy('annee','mois')\
     .toPandas().head(10) #.show()

,annee,mois,precipitations
0,NaN,NaN,11830.0
1,2024.0,NaN,11830.0
2,2024.0,1.0,1013.0
3,2024.0,2.0,938.0
4,2024.0,3.0,1001.0
5,2024.0,4.0,972.0
6,2024.0,5.0,1003.0
7,2024.0,6.0,967.0
8,2024.0,7.0,1006.0
9,2024.0,8.0,1011.0


In [16]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .rollup('annee','mois')\
     .agg(round(sum('precipitations')).alias('precipitations'))\
     .orderBy('annee','mois')\
     .toPandas().head(20) #.show(20)

,annee,mois,precipitations
0,NaN,NaN,11830.0
1,2024.0,NaN,11830.0
2,2024.0,1.0,1013.0
3,2024.0,2.0,938.0
4,2024.0,3.0,1001.0
5,2024.0,4.0,972.0
6,2024.0,5.0,1003.0
7,2024.0,6.0,967.0
8,2024.0,7.0,1006.0
9,2024.0,8.0,1011.0


In [17]:
meteo.where('id < 8000')\
     .rollup('annee','mois')\
     .agg(
            round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .toPandas().head(16) # .show(16)

,annee,mois,precipitations
0,2024.0,1.0,2171.0
1,2024.0,2.0,3374.0
2,2024.0,3.0,3811.0
3,2024.0,4.0,2144.0
4,2024.0,5.0,3992.0
5,2024.0,6.0,2243.0
6,2024.0,7.0,1708.0
7,2024.0,8.0,1782.0
8,2024.0,9.0,3791.0
9,2024.0,10.0,4355.0


In [18]:
meteo.where('id < 8000')\
     .rollup('annee','mois')\
     .agg(
            round(avg('temperature'),2).alias('temperature'),
            round(avg('humidite'),2).alias('humidite'),
            round(avg('visibilite'),2).alias('visibilite'),
            round(avg('pression'),2).alias('pression'),
            round(avg('precipitations'),2).alias('precipitations'))\
     .orderBy(col('annee').asc_nulls_last(),
              col('mois').asc_nulls_last())\
     .toPandas().head(16) #.show(16)

,annee,mois,temperature,humidite,visibilite,pression,precipitations
0,2024.0,1.0,5.90,0.82,21.56,99.88,0.21
1,2024.0,2.0,9.11,0.83,24.46,99.43,0.36
2,2024.0,3.0,10.03,0.78,27.49,98.79,0.38
3,2024.0,4.0,11.85,0.72,29.45,99.54,0.22
4,2024.0,5.0,14.84,0.78,25.83,99.29,0.40
5,2024.0,6.0,18.13,0.74,24.99,99.42,0.23
6,2024.0,7.0,20.93,0.72,26.98,99.48,0.17
7,2024.0,8.0,21.63,0.71,26.24,99.52,0.18
8,2024.0,9.0,16.41,0.78,27.05,99.44,0.39
9,2024.0,10.0,14.58,0.85,22.66,99.46,0.43


## cube
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-06.png" width="400">

In [19]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('precipitations'))\
     .cube('annee','mois')\
     .agg( round(sum('precipitations')).alias('precipitations'))\
     .orderBy(col('annee'),col('mois'))\
     .toPandas().head(24) #.show(24)

,annee,mois,precipitations
0,NaN,NaN,11830.0
1,NaN,1.0,1013.0
2,NaN,2.0,938.0
3,NaN,3.0,1001.0
4,NaN,4.0,972.0
5,NaN,5.0,1003.0
6,NaN,6.0,967.0
7,NaN,7.0,1006.0
8,NaN,8.0,1011.0
9,NaN,9.0,976.0


In [20]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            {'id':'count',
            'temperature':'avg',
            'humidite':'avg'}
     ).toPandas().head(10) #.show(10)

,id,annee,avg(humidite),avg(temperature),count(id)
0,7149,2024,0.782771,12.738094,2927
1,7020,2024,0.832407,12.893384,2924
2,7139,2024,0.828367,11.772087,2927
3,7661,2024,NaN,NaN,2885
4,7037,2024,0.841414,11.374034,2923
5,7130,2024,0.819668,12.536289,2921
6,7190,2024,0.793995,12.521975,2926
7,7072,2024,0.826489,11.957573,2925
8,7558,2024,0.739190,11.860570,2915
9,7577,2024,0.723352,14.673502,2921


In [21]:
meteo.where('id < 8000')\
     .groupBy('id','annee')\
     .agg(
            {'id':'count',
            'temperature':'avg',
            'humidite':'avg'}
     ).toDF('id','annee','humidite','temperature','nb_villes').toPandas().head(10) #.show(10)

,id,annee,humidite,temperature,nb_villes
0,7149,2024,0.782771,12.738094,2927
1,7020,2024,0.832407,12.893384,2924
2,7139,2024,0.828367,11.772087,2927
3,7661,2024,NaN,NaN,2885
4,7037,2024,0.841414,11.374034,2923
5,7130,2024,0.819668,12.536289,2921
6,7190,2024,0.793995,12.521975,2926
7,7072,2024,0.826489,11.957573,2925
8,7558,2024,0.739190,11.860570,2915
9,7577,2024,0.723352,14.673502,2921


In [22]:
meteo.where('id < 8000')\
     .groupBy('id')\
     .agg(
        round(skewness  ('temperature'),3).alias('skewness'  ),
        round(kurtosis  ('temperature'),3).alias('kurtosis'  ),
        round(variance  ('temperature'),3).alias('variance'  ),
        round(var_pop   ('temperature'),3).alias('var_pop'   ),
        round(stddev    ('temperature'),3).alias('stddev'    ),
        round(stddev_pop('temperature'),3).alias('stddev_pop'))\
     .orderBy('id')\
     .toPandas().head(15) #.show(15)

,id,skewness,kurtosis,variance,var_pop,stddev,stddev_pop
0,7005,0.070,0.427,34.289,34.277,5.856,5.855
1,7015,0.128,0.059,40.633,40.619,6.374,6.373
2,7020,-0.229,-0.276,13.898,13.893,3.728,3.727
3,7027,0.090,0.340,31.301,31.291,5.595,5.594
4,7037,0.111,0.185,35.756,35.743,5.980,5.979
5,7072,0.114,-0.012,48.257,48.240,6.947,6.946
6,7110,-0.072,0.508,19.828,19.821,4.453,4.452
7,7117,-0.273,0.217,16.225,16.220,4.028,4.027
8,7130,0.107,0.310,33.058,33.047,5.750,5.749
9,7139,0.142,0.209,37.853,37.840,6.152,6.151


In [23]:
meteo.where('id < 8000 and annee > 2014')\
     .groupBy('id','annee')\
     .agg( round(avg('temperature'),2).alias('temperature'))\
     .orderBy("id","annee")\
     .toPandas().head(10) #.show(10)

,id,annee,temperature
0,7005,2024,11.74
1,7015,2024,12.15
2,7020,2024,12.89
3,7027,2024,12.07
4,7037,2024,11.37
5,7072,2024,11.96
6,7110,2024,12.06
7,7117,2024,12.82
8,7130,2024,12.54
9,7139,2024,11.77


# pivot
<img src="https://raw.githubusercontent.com/rbizoi/AnalyserLesDonneesAvecSpark/main/DataFrameSpark/images/M06-04.png" width="400"> 

In [24]:
meteo.where('id < 8000 and annee > 2014')\
      .groupBy('id')\
      .pivot('annee')\
      .agg( round(avg('temperature'),2))\
      .sort('id')\
      .toPandas().head(10) #.show(10)

,id,2024
0,7005,11.74
1,7015,12.15
2,7020,12.89
3,7027,12.07
4,7037,11.37
5,7072,11.96
6,7110,12.06
7,7117,12.82
8,7130,12.54
9,7139,11.77


In [25]:
villes.select('ville',
               round('altitude',-2).alias('altitude'))\
      .groupBy('altitude')\
      .agg(collect_list('ville').alias('ville par altitude')).toPandas().head()
# .show(truncate=False)

,altitude,ville par altitude
0,300,"[NANCY-OCHEY, BALE-MULHOUSE, CLERMONT-FD, GOUR..."
1,900,[EMBRUN]
2,800,[LE PUY-LOUDES]
3,100,"[ABBEVILLE, CAEN-CARPIQUET, REIMS-PRUNAY, BRES..."
4,400,"[LIMOGES-BELLEGARDE, TARBES-OSSUN, ST GIRONS]"


In [26]:
meteo.where('id < 8000')\
     .groupBy('annee','mois')\
     .agg(round(sum('pression') / 1000).alias('pression'))\
     .orderBy('annee','mois')\
     .toPandas().head()

,annee,mois,pression
0,2024,1,1013.0
1,2024,2,938.0
2,2024,3,1001.0
3,2024,4,972.0
4,2024,5,1003.0
